# 金融・市場データの取得・可視化・分析



In [ ]:
#!pip install seaborn

In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得で用いるライブラリ
import json
import time
from urllib.parse import urljoin, urlencode
import requests
# データ処理に用いるライブラリ
import numpy as np
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

In [ ]:
load_dotenv()
FRED_API_KEY = os.getenv("FRED_API_KEY")
J_QUANTS_MAILADDRESS = os.getenv("J_QUANTS_MAILADDRESS")  # J-Quantsサインインのメールアドレス
J_QUANTS_PASSWORD = os.getenv("J_QUANTS_PASSWORD")  # J-Quantsサインインのパスワード

In [ ]:
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch09").resolve()

## 債券と金利

### 国債の残高と長期金利

- [国債](https://www.mof.go.jp/jgbs/index.html)
- [「統計表一覧（国債及び借入金並びに政府保証債務現在高）」のサイト](https://www.mof.go.jp/jgbs/reference/gbb/data.htm)
- [日本銀行の「時系列統計データ 検索サイト」](https://www.stat-search.boj.or.jp/)
- [「国債金利情報」のサイト](https://www.mof.go.jp/jgbs/reference/interest_rate/index.htm)
- [算出方法](https://www.mof.go.jp/jgbs/reference/interest_rate/outline.pdf)

#### 国債の残高

- [「統計表一覧（国債及び借入金並びに政府保証債務現在高）」のサイト](https://www.mof.go.jp/jgbs/reference/gbb/data.htm)

In [ ]:
bond_boj = pd.read_csv(
    data_path / "nme_政府債務合計_国債_内国債_暦年.csv",
    encoding="shift_jis",
    header=1, 
).rename(columns={"系列名称": "年", "＿種類別内訳/国債/内国債": "国債残高"})
bond_boj.tail()

#### 長期金利とイールド・カーブ


In [ ]:
mof_url = "https://www.mof.go.jp/jgbs/reference/interest_rate/"
csv_url = urljoin(mof_url, "jgbcm.csv")
all_csv_url = urljoin(mof_url, "data/jgbcm_all.csv")
#jgbcm_mof = pd.read_csv(csv_url, encoding="cp932", skiprows=1, index_col=0, na_values="-")  # サイトから直接読み込む
jgbcm_mof = pd.read_csv(data_path / "jgbcm.csv", encoding="cp932", skiprows=1, index_col=0, na_values="-")  # ダウンロードして読み込む
#jgbcm_all = pd.read_csv(all_csv_url, encoding="cp932", skiprows=1, index_col=0, na_values="-")  # サイトから直接読み込む
jgbcm_all = pd.read_csv(data_path / "jgbcm_all.csv", encoding="cp932", skiprows=1, index_col=0, na_values="-")  # ダウンロードして読み込む
jgbcm_concatenated = pd.concat([jgbcm_all, jgbcm_mof]).dropna(axis="index", how="all")
jgbcm_concatenated.index[:5], jgbcm_concatenated.index[5000:5005], jgbcm_concatenated.index[-5:]

In [ ]:
era_offsets = {"S": 1925, "H": 1988, "R": 2018}
ref_date_df = jgbcm_concatenated.index.to_series().str.extract(
    r"(?P<era>[SHR])(?P<year>\d+)\.(?P<month>\d+)\.(?P<day>\d+)",
    expand=True,
)
ce_index = pd.to_datetime({
    "year": ref_date_df["year"].astype(int) + ref_date_df["era"].map(era_offsets),
    "month": ref_date_df["month"].astype(int),
    "day": ref_date_df["day"].astype(int)
})
ce_index.name = jgbcm_concatenated.index.name
interest_rate = jgbcm_concatenated.copy().set_index(ce_index)
interest_rate

In [ ]:
# 各年の最終日の金利でテーブルを年次に整形
interest_rate_annual = interest_rate.resample("YE").last()
cond_yield = interest_rate_annual.index >= "2007-01-01"
cond_yield &= interest_rate_annual.index < "2025-01-01"
yield_data = (interest_rate_annual.loc[cond_yield]
    .stack()
    .reset_index()
    .rename(columns={"level_1": "maturity", 0: "rate"})
    .assign(**{
        "年": lambda df: df["基準日"].dt.strftime("%Y"),
        "年限": lambda df: df["maturity"].str.replace("年", "").astype(int),
        "金利": lambda df: df["rate"].astype(float),
    })
)
yield_data.tail()

In [ ]:
colors = px.colors.sequential.YlGn[::-1] + px.colors.sequential.Reds
yield_fig = px.line(yield_data,
    x="年限",
    y="金利",
    color="年",
    color_discrete_sequence=colors,
    markers=True,
)
yield_fig.update_layout(width=900, height=700, font={"size": 18},
    xaxis={"ticksuffix": "年"},
    yaxis={"ticksuffix": "%"},
)
yield_fig.show()

- [国債金利情報](https://www.mof.go.jp/jgbs/reference/interest_rate/index.htm)

### 短期金利

- [日本銀行の「時系列統計データ 検索サイト」](https://www.stat-search.boj.or.jp/index.html)
- [「主要時系列統計データ表」のページ](https://www.stat-search.boj.or.jp/ssi/mtshtml/fm02_m_1.html)


In [ ]:
jp_rate = pd.read_csv(data_path / "fm02_m_1.csv",
    encoding="cp932",
    skiprows=9,
    names=["date", "無担保コールレートO/N月平均金利"],
    usecols=[0, 2],
    parse_dates=["date"],
    date_format="%Y/%m",
    index_col="date",
)
jp_rate.tail()

### FRBとFFレート

- [FRED](https://fred.stlouisfed.org/)
- [FRED API](https://fred.stlouisfed.org/docs/api/fred/)
- [API利用規約](https://fred.stlouisfed.org/docs/api/terms_of_use.html)
- [プライバシーポリシー](https://www.stlouisfed.org/about-us/privacy-policy/online-notice)
- [FRED Graph](https://fred.stlouisfed.org/graph/?g=mQy7)

#### FREDのFFレート

- [FRED API の仕様](https://fred.stlouisfed.org/docs/api/fred/series_observations.html)


In [ ]:
fred_url = "https://api.stlouisfed.org/fred/series/observations"
series_mapping = {
    "DFF": "実効FF金利",
    "DFEDTARU": "上限金利",
    "DFEDTARL": "下限金利",
}
observation_start = "2007-01-01"
observation_end = "2024-12-31"
base_fred_params = {
    "api_key": FRED_API_KEY,
    "file_type": "json",
    "observation_start": observation_start,
    "observation_end": observation_end,
    "frequency": "m",  # Monthly
    "aggregation_method": "eop",  # End of period
}
fred_dfs = []
for series_id, rate_name in series_mapping.items():
    fred_params = base_fred_params.copy()
    fred_params["series_id"] = series_id
    fred_res = requests.get(fred_url, params=fred_params)
    fred_df = pd.DataFrame(fred_res.json()["observations"]).assign(**{
        "date": lambda df: pd.to_datetime(df["date"]),
        "value": lambda df: pd.to_numeric(df["value"], errors="coerce"),
        "金利種類": rate_name,
    })
    fred_dfs.append(fred_df)
    time.sleep(5)
ffrate = pd.concat(fred_dfs)
ffrate

In [ ]:
#ffrate.to_csv(data_path / "ch09_fred_ffrate.csv")

In [ ]:
fred_id = "DFF"
fredgraph_df = pd.read_csv(f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={fred_id}")

In [ ]:
#fredgraph_df.to_csv(data_path / "ch09_fred_fredgraph_df.csv")

In [ ]:
ff_fig = px.line(ffrate, x="date", y="value", color="金利種類")
ff_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"ticksuffix": "%", "title": None},
)
ff_fig.show()

#### 日米金利


In [ ]:
fedfunds_params = base_fred_params.copy()
fedfunds_params["series_id"] = "FEDFUNDS"  # 月次
fedfunds_params["observation_start"] = "1985-01-01"
fedfunds_res = requests.get(fred_url, params=fedfunds_params)
fedfunds_df = pd.DataFrame(fedfunds_res.json()["observations"]).assign(**{
    "date": lambda df: pd.to_datetime(df["date"]),
    "米国": lambda df: pd.to_numeric(df["value"], errors="coerce"),
})[["date", "米国"]]
fedfunds_df

In [ ]:
rate_merged = fedfunds_df.merge(
    jp_rate, left_on="date", right_index=True, how="left"
).rename(columns={"無担保コールレートO/N月平均金利": "日本"})
rate_tidy = pd.melt(rate_merged,
    id_vars=["date"],
    value_vars=["米国", "日本"],
    var_name="国",
    value_name="利率",
    ignore_index=False,
).reset_index()
rate_fig = px.line(rate_tidy, x="date", y="利率", color="国")
rate_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"ticksuffix": "%"},
)
rate_fig.show()

## 外国為替

- [名目実効為替レート](https://www.boj.or.jp/statistics/outline/exp/exrate02.htm)
- [実効為替レート](https://www.bis.org/statistics/eer.htm)

### FREDの為替データ

- [FRED のサイト](https://fred.stlouisfed.org/)


In [ ]:
fx_params = base_fred_params.copy()
fx_params["series_id"] = "DEXJPUS"
fx_params["observation_start"] = "1985-01-01"
fx_params["observation_end"] = "2024-12-31"
fx_res = requests.get(fred_url, params=fx_params)
fx_df = pd.DataFrame(fx_res.json()["observations"]).assign(**{
    "date": lambda df: pd.to_datetime(df["date"]),
    "value": lambda df: pd.to_numeric(df["value"], errors="coerce"),
})[["date", "value"]]

In [ ]:
#fx_df.to_csv(data_path / "ch09_fred_fx.csv")
#fx_df = pd.read_csv(data_path / "ch09_fred_fx.csv", index_col=0)

In [ ]:
# 日米金利差を計算
rate_wide = rate_merged.assign(**{"金利差": rate_merged["米国"] - rate_merged["日本"]})
parity_fig = go.Figure()
# ドル円レートのデータを追加
parity_fig.add_trace(
    go.Scatter(x=fx_df["date"], y=fx_df["value"], name="USD/JPY", marker_color="blue")
)
# 右側のy軸に金利差のデータを追加
parity_fig.add_trace(go.Scatter(
    x=rate_wide["date"],
    y=rate_wide["金利差"],
    name="金利差",
    marker_color="red",
    yaxis="y2",
))
parity_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None}, 
    yaxis={"title": "為替レート(円)"},
    yaxis2={"title": "金利差(%)", "overlaying": "y", "side": "right"}
)
parity_fig.show()

### 国際決済銀行の為替データ

- [国際決済銀行](https://www.bis.org/)
- [BIS statistics](https://www.bis.org/statistics/index.htm)
- [BIS Data Portal](https://data.bis.org/)
- [BIS SDMX service API](https://www.bis.org/statistics/sdmx_techspec.htm)
- [BIS SDMX RESTful API](https://stats.bis.org/api-doc/v2/#/)
- [Bulk downloads](https://data.bis.org/bulkdownload)
- [二国間為替レート：bilateral exchange rates](https://data.bis.org/topics/XRU)


In [ ]:
# Timespanを2015-01-01から2024-12-31に絞る
start_period = "2015-01-01"
end_period = "2024-12-31"
# Frequencyを月次に絞る
frequency = "M"
# Currencyを米ドル、ユーロ(エリアはドイツ)、日本円、英国ポンド、豪ドル、カナダドルに絞る
currency_list = ["US", "DE", "JP", "GB", "AU", "CA"]
# COLLECTION IndicatorをEnd of Periodに絞る
collection_indicator = "E"
# Request URLを作成してCSVを読み込み
currency = "+".join(currency_list)
bis_base_url = "https://stats.bis.org/api/v2/data/dataflow/BIS/WS_XRU/1.0/"
bis_endpoint = f"{frequency}.{currency}..{collection_indicator}"
bis_endpoint_url = urljoin(bis_base_url, bis_endpoint)
bis_params = {"startPeriod": start_period, "endPeriod": end_period, "format": "csv"}
bis_request_url = urljoin(bis_endpoint_url, "?" + urlencode(bis_params))
usecols = ["FREQ", "REF_AREA", "CURRENCY", "COLLECTION", "TIME_PERIOD", "OBS_VALUE"]
bis_fx_df = pd.read_csv(
    bis_request_url, usecols=usecols, parse_dates=["TIME_PERIOD"], date_format="%Y-%m"
)
bis_fx_df

In [ ]:
#bis_fx_df.to_csv(data_path / "ch09_bis_fx.csv")
#bis_fx_df = pd.read_csv(data_path / "ch09_bis_fx.csv", index_col=0)

In [ ]:
fx_wide = (bis_fx_df[["TIME_PERIOD", "CURRENCY", "OBS_VALUE"]]
    .set_index(["TIME_PERIOD", "CURRENCY"])
    .unstack()
)
fx_change = fx_wide / fx_wide.iloc[0, :]
fx_tidy = (fx_change
    .stack(future_stack=True)
    .reset_index()
    .rename(columns={"CURRENCY": "通貨", "OBS_VALUE": "実効為替レート"})
)
fx_fig = px.line(
    fx_tidy, x="TIME_PERIOD", y="実効為替レート", color="通貨", width=800, height=450
)
fx_fig.update_layout(font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
)
fx_fig.show()

## マネー

### マネーストック

#### 日本銀行と日本のマネーストック


In [ ]:
money_boj = pd.read_csv(data_path / "md02_m_1.csv", encoding="shift_jis", header=3)
money_boj.iloc[:10, [0] + list(range(7, 11))]

In [ ]:
moneystock = money_boj.loc[5:, :]
moneystock = moneystock.assign(**{
    "年月": pd.to_datetime(moneystock["系列名称"], format="%Y/%m"),
    "年": moneystock["系列名称"].str[:4].astype(int),
    "マネーストック": moneystock["Ｍ２／平／マネーストック"].astype(float)
})
# 期末に絞る
cond_m2 = moneystock["年月"].dt.month == 12
cond_m2 &= moneystock["年"] > 2004
m2 = moneystock.loc[cond_m2, ["年", "マネーストック"]]
# 国債残高と結合
bond_m2 = bond_boj.merge(m2, on="年")[["年", "国債残高", "マネーストック"]]
bond_m2.tail()

In [ ]:
# 整然データ形式に変換
m2_tidy = pd.melt(bond_m2,
    id_vars="年",
    value_vars=["国債残高", "マネーストック"],
    var_name="残高",
    value_name="金額(兆円)",
)
# 兆円単位に変換
m2_tidy = m2_tidy.assign(**{"金額(兆円)": m2_tidy["金額(兆円)"] / 10000})
# 棒グラフで可視化
m2_fig = px.bar(m2_tidy, x="年", y="金額(兆円)", color="残高", barmode="group")
m2_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ".0f", "ticksuffix": "兆円", "title": None},
)
m2_fig.show()

#### FRBと米国のマネーストック

- [FRED](https://fred.stlouisfed.org/)


In [ ]:
m2_params = base_fred_params.copy()
m2_params["observation_start"] = "1990-01-01"
m2_params["observation_end"] = "2024-12-31"
m2_params["frequency"] = "a"  # Annual
m2_series_mapping = {
    "M2SL": "M2(マネーストック)",
    "GDP": "名目GDP",
}
m2_fred_dfs = []
for series_id, series_name in m2_series_mapping.items():
    fred_params = m2_params.copy()
    fred_params["series_id"] = series_id
    fred_res = requests.get(fred_url, params=fred_params)
    fred_df = pd.DataFrame(fred_res.json()["observations"]).assign(**{
        "日付": lambda df: pd.to_datetime(df["date"]),
        "value": lambda df: pd.to_numeric(df["value"], errors="coerce"),
        "系列名": series_name,
    })
    m2_fred_dfs.append(fred_df)
m2_fred_df = pd.concat(m2_fred_dfs)
m2_gdp_pivot = m2_fred_df.pivot_table(values="value", index="日付", columns="系列名")
m2_gdp = m2_gdp_pivot.assign(**{
    "m2velocity": m2_gdp_pivot["名目GDP"] / m2_gdp_pivot["M2(マネーストック)"],
    "marshall_k": m2_gdp_pivot["M2(マネーストック)"] / m2_gdp_pivot["名目GDP"]
})
m2_gdp.tail()

In [ ]:
m2_gdp_tidy = m2_fred_df[["日付", "系列名", "value"]]
m2_gdp_fig = px.line(m2_gdp_tidy, x="日付", y="value", color="系列名")
m2_gdp_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "十億ドル", "title": None}
)
m2_gdp_fig.show()

### 資金循環統計

- [資金循環統計サイト](https://www.boj.or.jp/statistics/sj/index.htm)


In [ ]:
cols_boj = [
    "系列名称",
    "負債・資金過不足／中央銀行／フロー",
    "負債・資金過不足／預金取扱機関／フロー",
    "負債・資金過不足／民間非金融法人企業／フロー",
    "負債・資金過不足／一般政府／フロー",
    "負債・資金過不足／うち公的年金／フロー",
    "負債・資金過不足／家計／フロー",
    "負債・資金過不足／海外／フロー",
]
fund_flow_df = pd.read_csv(data_path / "nme_資金循環_資金過不足_年度.csv",
    header=1,
    encoding="shift_jis",
    usecols=cols_boj,
)
fund_flow_df.head(3).T

In [ ]:
fund_flow_tidy = (fund_flow_df
    .set_index("系列名称")
    .stack()
    .reset_index()
    .rename(columns={"level_1": "部門", 0: "資金過不足"})
)
fund_flow_tidy = fund_flow_tidy.assign(**{
    "部門": lambda df: df["部門"].str.split("／", expand=True).iloc[:, 1]
})
fund_flow_fig = px.line(
    fund_flow_tidy, x="系列名称", y="資金過不足", color="部門", title="資金過不足"
)
fund_flow_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "億円", "title": None},
)
fund_flow_fig.show()

## 株式と株価

- [日本取引所グループ](https://www.jpx.co.jp/)
- [ニューヨーク証券取引所](https://www.nyse.com/index)
- [NYSE American](https://www.nyse.com/markets/nyse-mkt)
- [TOPIX](https://www.jpx.co.jp/markets/indices/topix/)
- [日経平均株価](https://indexes.nikkei.co.jp/nkave/index/profile?idx=nk225)

### 有価証券報告書データと金融庁のEDINET

- [EDINET（Electronic Disclosure for Investors Network）閲覧サイト](https://disclosure2.edinet-fsa.go.jp/week0010.aspx)
- [XBRL Japan](https://www.xbrl.or.jp/)
- [Arelle](https://arelle.org/arelle/)
- [arellerelease](https://arelle.readthedocs.io/en/2.23.4/)
- [EDINET の利用規約](https://disclosure2dl.edinet-fsa.go.jp/guide/static/disclosure/WZEK0030.html)
- [XBRL 作成ガイドライン](https://www.fsa.go.jp/search/20231211.html)

#### 資本金別企業数


In [ ]:
edi_info = pd.read_csv(data_path / "EdinetcodeDlInfo.csv", encoding="cp932", skiprows=1)
edi_info.head(3).T

In [ ]:
cond_edi = edi_info["提出者種別"] == "内国法人・組合"
cond_edi &= edi_info["上場区分"] == "上場"
edi_df = edi_info.loc[cond_edi]
edi_df = edi_df.assign(**{
    "資本金区分": (edi_df["資本金"].apply(np.log10).astype(int)+1).astype(str) + "桁(百万円)"
})
cols_edi = ["資本金区分", "提出者業種"]
edi_crosstab = (edi_df[cols_edi]
    .groupby(cols_edi, dropna=False, as_index=False)
    .size()
    .rename(columns={"提出者業種": "業種", "size": "企業数"})
)
edi_crosstab

In [ ]:
edi_fig = px.bar(edi_crosstab,
    x="業種",
    y="企業数",
    color="資本金区分",
    color_discrete_sequence=plotly.colors.sequential.Mint,
)
edi_fig.update_layout(width=1000, height=600, font={"size": 18})
edi_fig.show()

#### 損益計算書ウォーターフォールチャート

- [トヨタ自動車の有価証券報告書のPDF](https://global.toyota/jp/ir/library/securities-report/)


In [ ]:
ufo_edi = pd.read_csv(data_path / "jpcrp_トヨタ自動車_202403.csv", 
    encoding="utf-16", sep="\t", na_values={"値": "－"})
ufo_edi.columns

In [ ]:
cond_pl = ufo_edi["相対年度"] == "当期"
cond_pl &= ufo_edi["連結・個別"] == "個別"
cond_pl &= ufo_edi["コンテキストID"] == "CurrentYearDuration_NonConsolidatedMember"
pl_df = ufo_edi.loc[cond_pl, ["要素ID", "項目名", "単位", "値"]]
pl_df.iloc[10:19, :]

In [ ]:
# 「（」以下の「△」などの項目を除外する
pl_df = pl_df.assign(**{"item": pl_df["項目名"].str.replace("（.*", "", regex=True)})
pl_df = pl_df.assign(**{"value": pl_df["値"].fillna(0).astype(int)})
pl_data = [
    ["売上高", "relative", 1],
    ["売上原価", "relative", -1],
    ["売上総利益又は売上総損失", "total", 1],
    ["販売費及び一般管理費", "relative", -1],
    ["営業利益又は営業損失", "total", 1],
    ["営業外収益", "relative", 1],
    ["営業外費用", "relative", -1],
    ["経常利益又は経常損失", "total", 1],
    ["法人税等", "relative", -1],
    ["当期純利益又は当期純損失", "total", 1],
]
statements, measures, signs = zip(*pl_data)
waterfall_data = pd.DataFrame(pl_data, columns=["statement", "measure", "sign"])
# 各PL項目に表示する金額テキストは、
# 各項目名で抽出した"value"は1000000で除算して100万円単位にする
waterfall_data = waterfall_data.assign(**{"value": waterfall_data["statement"].apply(
        lambda label: pl_df.set_index("item").loc[label, "value"] // 1_000_000
    ) * waterfall_data["sign"]
})
waterfall_data

In [ ]:
waterfall_fig = go.Figure(go.Waterfall(
    x=statements,
    y=waterfall_data["value"],
    measure=measures,  # 各棒の計測タイプを指定
    text=waterfall_data["value"].astype(str), 
    textposition="outside"  # テキストを棒の外側に配置
))
# レイアウトでy軸の範囲を0円から売上より少し大きい金額までに設定
waterfall_fig.update_layout(width=900, height=450,
    yaxis={
        "tickformat": ",.0f",
        "ticksuffix": "百万円",
        "title": None,
        "range": [
            0,
            int(waterfall_data.set_index("statement").at["売上高", "value"])
            * 1.15,
        ],
    },
)
waterfall_fig.show()

#### 貸借対照表

- [2024年版EDINETタクソノミの公表について](https://www.fsa.go.jp/search/20231211.html)


In [ ]:
cond_bs = ufo_edi["項目名"].isin(["固定資産", "流動資産", "純資産", "固定負債", "流動負債"])
cond_bs &= ufo_edi["相対年度"].isin(["前期末", "当期末"])
cond_bs &= ufo_edi["連結・個別"] == "個別"
cond_bs &= ufo_edi["コンテキストID"].isin(["CurrentYearInstant_NonConsolidatedMember", 
    "Prior1YearInstant_NonConsolidatedMember"])
balance = ufo_edi.loc[cond_bs].drop_duplicates()
# 金額を百万単位に変換
balance = balance.assign(**{"value": balance["値"].replace("－", "0").astype(int) // 1_000_000})

In [ ]:
# 相対年度、項目名、値の列に絞ったテーブルを作成
bs_df = balance.pivot(index="相対年度", columns="項目名", values="value")
# 前期と当期の差額
diff_val = (bs_df.loc["当期末", "流動資産"] + bs_df.loc["当期末", "固定資産"]) - (
    bs_df.loc["前期末", "流動資産"] + bs_df.loc["前期末", "固定資産"]
)
# グラフ設定のための定数を用意
bar_width = 0.4  # 棒の幅の設定用
xpos = {"前期末": 0, "当期末": 1}  # 棒のx軸の位置設定用
cols_bs = ["固定資産", "流動資産", "純資産", "固定負債", "流動負債"]
# 資産が青系、負債が赤系、純資産が緑になるように色を設定
bs_colors = {
    "固定資産": "#1f77b4",
    "流動資産": "#aec7e8",
    "純資産": "#2ca02c",
    "固定負債": "#d62728",
    "流動負債": "#ff9896",
}
# 資産側(左)と負債・純資産側(右)に分ける
offsetgrps = {"固定資産": 0, "流動資産": 0, "純資産": 1, "固定負債": 1, "流動負債": 1}
# 積上棒グラフの開始位置に使用
base_dict = {
    "流動資産": bs_df["固定資産"],
    "固定負債": bs_df["純資産"],
    "流動負債": bs_df["純資産"] + bs_df["固定負債"],
}
# グラフ描画領域の作成
bs_fig = go.Figure()
# 各棒グラフの追加
for col in cols_bs:
    base = base_dict[col] if col in base_dict else None
    bs_fig.add_trace(go.Bar(
        name=col,
        x=[xpos[i] for i in bs_df.index],  # 棒のx軸の位置
        y=bs_df[col],  # 棒の底辺からの高さ
        marker_color=bs_colors[col],  # 棒の色
        offsetgroup=offsetgrps[col],  # ズレを同じグループ内で揃える
        base=base,  # 棒の底辺の開始位置
        width=bar_width,  # 棒の幅
        text=[f"{col}:{val}百万円" for val in bs_df[col]],  # 棒に表示するテキスト
        textposition="inside",  # テキストの位置を棒の内側に設定
        showlegend=False,  # 凡例を表示しない
    ))
# 前期と当期の変化を明示する線を追加
bs_fig.add_trace(go.Scatter(
    x=[0 + bar_width, 1 - bar_width],
    y=bs_df["流動資産"] + bs_df["固定資産"],
    line={"color": "black", "width": 4, "dash": "dash"},
    mode="lines",
    showlegend=False,
))
# レイアウトの更新
bs_fig.update_layout(width=1000, height=500,
    barmode="group",
    # x軸の0と1の値の位置にそれぞれ"前期末"と"当期末"というテキストを設定
    xaxis={"tickvals": [0, 1], "ticktext": ["前期末", "当期末"]},
    yaxis={"tickformat": ",.0f", "ticksuffix": "百万円"},
    # 前期と当期の差額の金額をアノテーションを用いて表示
    annotations=[
        {
            "x": (xpos["前期末"] + xpos["当期末"]) / 2,
            "y": (bs_df.loc["当期末", "流動資産"] + bs_df.loc["当期末", "固定資産"])
            * 1.1,
            "text": str(diff_val) + "百万円",
            "font": {"size": 15},
            "showarrow": False,  # アノテーションを指す矢印を表示しない
        }
    ],
)
bs_fig.show()

### 株価と日本取引所(JPX)のJ-Quants

- [J-Quants API](https://jpx-jquants.com/)
- [API 仕様](https://jpx.gitbook.io/j-quants-ja/api-reference)
- [サービス利用規約](https://jpx-jquants.com/termsofservice)

#### J-Quants


In [ ]:
jq_payload = {"mailaddress": J_QUANTS_MAILADDRESS, "password": J_QUANTS_PASSWORD}
refresh_url = "https://api.jquants.com/v1/token/auth_user"
res_refresh = requests.post(refresh_url, data=json.dumps(jq_payload))
REFRESH_TOKEN = res_refresh.json()["refreshToken"]

In [ ]:
id_url = f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}"
res_id = requests.post(id_url)
id_token = res_id.json()["idToken"]

In [ ]:
jq_url = "https://api.jquants.com/v1/prices/daily_quotes"
jq_headers = {"Authorization": f"Bearer {id_token}"}
# 過去2年分の12週間遅延に合わせて日付を設定してください
#jq_params = {"code": "3994", "from": "20231201", "to": "20250630"}
jq_params = {"code": "3994", "from": "ここを設定", "to": "ここを設定"}
res_quotes = requests.get(jq_url, params=jq_params, headers=jq_headers)

In [ ]:
price_df = pd.json_normalize(res_quotes.json(), record_path=["daily_quotes"])
price_df.columns

In [ ]:
#price_df.to_csv("ch09_jquants_price.csv")

#### 株価とローソク足


In [ ]:
ohlc_fig = go.Figure(data=go.Ohlc(
    x=price_df["Date"],
    open=price_df["Open"],
    high=price_df["High"],
    low=price_df["Low"],
    close=price_df["Close"],
))
ohlc_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"tickformat": "%Y-%m"},
    yaxis={"tickformat": ",.0f", "ticksuffix": "円"},
)
ohlc_fig.show()

### 米国株の株価

- [YahooFinance](https://finance.yahoo.com/)
- [Stooq](https://stooq.com/)
- [Quandl](https://data.nasdaq.com/publishers/QDL)
- [Alpha Vantage](https://www.alphavantage.co/)
- [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/)

## 金融市場の指数

### 株価指数

- [TOPIX（東証株価指数）](https://www.jpx.co.jp/markets/indices/topix/)
- [TOPIXの見直し](https://www.jpx.co.jp/markets/indices/revisions-indices/index.html)
- [東証指数算出要領](https://www.jpx.co.jp/markets/indices/line-up/files/cal2_30_topix.pdf)
- [日経平均株価](https://indexes.nikkei.co.jp/nkave/index/profile?idx=nk225)
- [S&P 500（Standard & Poorʼs 500 Stock Index）](https://www.spglobal.com/spdji/jp/indices/equity/sp-500/#overview)
- [S&P ダウ・ジョーンズ・インデックス](https://www.spglobal.com/spdji/jp/)
- [S&P 米国株価指数 メソドロジー](https://www.spglobal.com/spdji/jp/documents/methodologies/methodology-sp-us-indices-japanese.pdf)
- [ダウ・ジョーンズ工業株価平均（Dow Jones Industrial Average：DJIA）](https://www.spglobal.com/spdji/jp/indices/equity/dow-jones-industrial-average/)
- [NASDAQ 総合指数（NASDAQ Composit Index）](https://www.nasdaq.com/solutions/global-indexes/nasdaq-composite)
- [NASDAQ Global Index Data Service（GIDS）](https://www.nasdaq.com/solutions/global-indexes/data/gids)
- [CRSP US トータル・マーケット・インデックス](https://www.crsp.org/indexes/crsp-u-s-total-market-index/)
- [Center for Research in Security Prices](https://www.crsp.org/)
- [Wilshire 5000 トータル・マーケット・インデックス](https://www.wilshireindexes.com/products/ft-wilshire-5000-index-series)
- [Wilshire Associates](https://www.wilshireindexes.com/)
- [FRED Will Remove Wilshire Index Data on June 3, 2024](https://news.research.stlouisfed.org/2024/04/fred-will-remove-wilshire-index-data-on-june-3-2024/)
- [MSCI](https://www.msci.com/home)
- [FTSE グローバル・オールキャップ・インデックス](https://www.ftserussell.com/ja/products/indices/geisac)
- [FTSEグループ](https://www.lseg.com/en/ftse-russell)


In [ ]:
fred_url = "https://api.stlouisfed.org/fred/series/observations"
series_list = ["SP500", "DJIA", "NASDAQCOM"]
stock_observation_start = "2016-01-01"  # 過去10年以内の日付
stock_observation_end = "2025-06-30"
base_fred_stock_params = {
    "api_key": FRED_API_KEY,
    "file_type": "json",
    "observation_start": stock_observation_start,
    "observation_end": stock_observation_end,
    "frequency": "m",  # Monthly
    "aggregation_method": "eop",  # End of period
}
fred_stock_dfs = []
for series_id in series_list:
    fred_params = base_fred_stock_params.copy()
    fred_params["series_id"] = series_id
    fred_res = requests.get(fred_url, params=fred_params)
    fred_df = pd.DataFrame(fred_res.json()["observations"]).assign(**{
        "日付": lambda df: pd.to_datetime(df["date"]),
        "value": lambda df: pd.to_numeric(df["value"], errors="coerce"),
        "Index": series_id,
    })
    fred_stock_dfs.append(fred_df)
    time.sleep(5)
us_stock = pd.concat(fred_stock_dfs)

In [ ]:
us_ts = us_stock.pivot_table(
    index="日付",
    columns="Index",
    values="value",
).dropna(axis=0, how="any")
monthly_ts = us_ts.resample("ME").last()  # Month End(月末)
monthly_pct = monthly_ts.pct_change().dropna()
sns.heatmap(monthly_pct.corr(), square=True, annot=True)

In [ ]:
ts_wide = us_ts / us_ts.iloc[0, :]
ts_melt = pd.melt(ts_wide, 
    value_vars=series_list, 
    var_name="指数", 
    value_name="リターン", 
    ignore_index=False
)
ts_tidy = ts_melt.reset_index()
us_stock_fig = px.line(ts_tidy, x="日付", y="リターン", color="指数")
us_stock_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年"},
)
us_stock_fig.show()

### 株価指数以外の金融市場の指数

- [NOMURA-BPI](http://qr.nomura.co.jp/jp/bpi/index.html)
- [FTSE 世界国債インデックス](https://www.ftserussell.com/ja/products/indices/world-government-bond-index)

### 投資信託のデータと金融市場の指数

- [投資信託協会](https://www.toushin.or.jp/index.html)
- [投信総合検索ライブラリー](https://toushin-lib.fwg.ne.jp/FdsWeb/)
